# What drives the price of a car?

![](images/kurt.jpeg)

### Business Understanding

Using a dataset with over 400,000 used car sale prices, we will try to uncover the most important attributes that drive the sale of used car. We will apply several models and search for the best hyperparameters for the models which produce the most accurate results. To validate our models we will keep a set of holdout data that is not provided to the model training phase.

### Data Understanding

In order to get more familiar with our data we will need to look at a few key aspects:

1. Which fields are available for consideration.
2. Which fields are missing information, and can we provide a default value for the missing data.
3. Which information can be ignored and dropped from our analysis.

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import train_test_split

In [2]:
vehicles = pd.read_csv('data/vehicles.csv')
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [3]:
# We can look at the various categorical columns individually to see what data they
# contain and the numbers of vehicles of each category. For instance the cylinders 
# column may be a good proxy for horsepower, and it might be convertible to a numeric
# value. However there are a lot of missing values and the 'other' value would throw
# off any regression that tries to predict price from cylinders if we encode 'other' as
# either zero or a large number. This will have to be one-hot and filled with 'other'.
vehicles[['cylinders']].value_counts()

cylinders   
6 cylinders     94169
4 cylinders     77642
8 cylinders     72062
5 cylinders      1712
10 cylinders     1455
other            1298
3 cylinders       655
12 cylinders      209
Name: count, dtype: int64

In [13]:
# Condition could be turned into a set of one-hot encoded columns with 'unknown'
# filled in for cars that are missing the data. It would be better to turn this 
# into an ordinal value as it clearly increases in qualityfrom salvage to new.
vehicles[['condition']].value_counts()

condition
good         121235
excellent     99749
like new      21178
fair           6769
new            1303
salvage         601
Name: count, dtype: int64

In [12]:
vehicles[['title_status']].value_counts()

title_status
clean           235738
unknown           5483
rebuilt           5168
salvage           2488
lien              1315
missing            515
parts only         128
Name: count, dtype: int64

In [5]:
# The model data is very dirty with values that include flame emoji. It also has a
# very high cardinality of 28,567 unique values. This isn't extremely useful for us.
vehicles[['model']].value_counts()

model                      
f-150                          8009
silverado 1500                 5140
1500                           4211
camry                          3135
silverado                      3023
                               ... 
crosstrek 2.0i premium aw         1
crosstrek 2.0i prem               1
crosstrek 2.0i limited cvt        1
crosstrek 2.0i limited aw         1
🔥GMC Sierra 1500 SLE🔥 4X4 🔥       1
Name: count, Length: 29649, dtype: int64

### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [8]:
# From the info method, we can see that there are many missing values for key 
# attributes of model year, fuel, condition, and odometer. We can drop rows missing
# values in any key column so that we have data on each car for those values.
vehicles.dropna(subset = ['odometer', 'year', 'fuel', 'condition'], inplace=True)
print(f"Number of vehicles with key statistics: {vehicles.shape[0]}")

Number of vehicles with key statistics: 250835


In [9]:
# We can drop several columns with high cardinality or low quality.
vehicles.drop(['VIN', 'model', 'size'],axis = 1, inplace=True)

In [10]:
# Fill in the missing values for cylinders, manufacturer, etc.
# Some of these already had an "other" value so we use that instead of unknown.
vehicles['manufacturer'] = vehicles['manufacturer'].fillna('unknown')
vehicles['cylinders'] = vehicles['cylinders'].fillna('other')
vehicles['title_status'] = vehicles['title_status'].fillna('unknown')
vehicles['transmission'] = vehicles['transmission'].fillna('other')
vehicles['drive'] = vehicles['drive'].fillna('unknown')
vehicles['type'] = vehicles['type'].fillna('other')
vehicles['paint_color'] = vehicles['paint_color'].fillna('unknown')

In [11]:
# Convert condition to ordinal.

<class 'pandas.core.frame.DataFrame'>
Index: 250835 entries, 27 to 426879
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            250835 non-null  int64  
 1   region        250835 non-null  object 
 2   price         250835 non-null  int64  
 3   year          250835 non-null  float64
 4   manufacturer  250835 non-null  object 
 5   condition     250835 non-null  object 
 6   cylinders     250835 non-null  object 
 7   fuel          250835 non-null  object 
 8   odometer      250835 non-null  float64
 9   title_status  250835 non-null  object 
 10  transmission  250835 non-null  object 
 11  drive         250835 non-null  object 
 12  type          250835 non-null  object 
 13  paint_color   250835 non-null  object 
 14  state         250835 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 30.6+ MB


### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.